# Import and Initialize

In [1]:
from mxESP32Debugger.debugger import Dbg as MXDBG
import time
from tqdm import trange
from loguru import logger

dev = MXDBG()

2025-04-10 16:36:40.763 | INFO     | mxESP32Debugger.debugger:connect:419 - Using serial port to connect. Port: COM4
2025-04-10 16:36:40.763 | INFO     | mxESP32Debugger.debugger:connect:424 - Using ESP32-S3R8.
2025-04-10 16:36:40.764 | INFO     | mxESP32Debugger.debugger:connect:425 - Embedded software version: v1.0
2025-04-10 16:36:40.764 | INFO     | mxESP32Debugger.debugger:connect:426 - Library version: v241210.0.rc3.


# I2C Init

Hardware I2C：
- HW_I2C_0: sdio 10 / sclk 11
- HW_I2C_1: sdio 41 / sclk 42

Soft I2C:
- SW_I2C_0: sdio 39 / sclk 40
- SW_I2C_1: sdio 37 / sclk 38
- SW_I2C_2: sdio 35 / sclk 36
- SW_I2C_3: sdio 33 / sclk 34
- SW_I2C_4: sdio 1 / sclk 2
- SW_I2C_5: sdio 4 / sclk 5
- SW_I2C_6: sdio 7 / sclk 8
- SW_I2C_7: sdio 17 / sclk 18 # which may be conflict with PWMs' pin definition


![The board and the ports](board_and_ports.png)

In [2]:
soft_i2c_groups_dict = {
    '0': {'sda': 39, 'scl': 40},
    '1': {'sda': 37, 'scl': 38},
    '2': {'sda': 35, 'scl': 36},
    '3': {'sda': 33, 'scl': 34},
    '4': {'sda': 1, 'scl': 2},    
    '5': {'sda': 4, 'scl': 5},
    '6': {'sda': 7, 'scl': 8},
}

for group, pins in soft_i2c_groups_dict.items():
    dev.soft_i2c_config(port=int(group), sda_pin=pins['sda'], scl_pin=pins['scl'], freq=400_000, pullup_enable=True)
    
ret, data = dev.i2c_config(sda_pin=10, scl_pin=11, freq=400000, sda_pullup=True, scl_pullup=True, port=0)
print(ret, data)
ret, data = dev.i2c_config(sda_pin=41, scl_pin=42, freq=400000, sda_pullup=True, scl_pullup=True, port=1)
print(ret, data)

True None
True None


In [3]:
# software i2c test
for port in range(7):
    ret, data = dev.soft_i2c_write_read(slave_id=0x78, write_list=[0x00], read_length=2, port=port)
    if ret is False:
        logger.error("Return {} in SW I2C port {}".format(ret, port))
        continue
    logger.info("SW I2C Port {} data: 0x{:04X}".format(port, data[0] << 8 | data[1]))

# hardware i2c test (port 0)
ret, data = dev.i2c_write_read(slave_id=0x78, write_list=[0x00], read_length=2, port=0)
if ret != False:
    logger.info("HW I2C Port 0 data: 0x{:04X}".format(data[0] << 8 | data[1]))
else:
    logger.error("Return {} in HW I2C port {}".format(ret, 0))

# hardware i2c test (port 1)
ret, data = dev.i2c_write_read(slave_id=0x38, write_list=[0x00], read_length=2, port=1)
if ret != False:
    logger.info("HW I2C Port 1 data: 0x{:04X}".format(data[0] << 8 | data[1]))
else:
    logger.error("Return {} in HW I2C port {}".format(ret, 1))
    
# hardware i2c test (port 1, different slave id)
ret, data = dev.i2c_write_read(slave_id=0x5D, write_list=[0x00], read_length=2, port=1)
if ret != False:
    logger.info("HW I2C Port 1 data: 0x{:04X}".format(data[0] << 8 | data[1]))
else:
    logger.error("Return {} in HW I2C port {}".format(ret, 1))

2025-04-10 16:36:41.147 | INFO     | __main__:<module>:7 - SW I2C Port 0 data: 0x0400
2025-04-10 16:36:41.159 | ERROR    | mxESP32Debugger.debugger:__check_ret_code:809 - Error code: 0x01002104, Description: SOFT I2C write read failed
2025-04-10 16:36:41.159 | ERROR    | __main__:<module>:5 - Return False in SW I2C port 1
2025-04-10 16:36:41.171 | INFO     | __main__:<module>:7 - SW I2C Port 2 data: 0x0400
2025-04-10 16:36:41.182 | INFO     | __main__:<module>:7 - SW I2C Port 3 data: 0x0400
2025-04-10 16:36:41.194 | INFO     | __main__:<module>:7 - SW I2C Port 4 data: 0x0400
2025-04-10 16:36:41.205 | INFO     | __main__:<module>:7 - SW I2C Port 5 data: 0x0400
2025-04-10 16:36:41.217 | INFO     | __main__:<module>:7 - SW I2C Port 6 data: 0x0400
2025-04-10 16:36:41.228 | INFO     | __main__:<module>:12 - HW I2C Port 0 data: 0x0400
2025-04-10 16:36:41.241 | INFO     | __main__:<module>:19 - HW I2C Port 1 data: 0x1800
2025-04-10 16:36:41.253 | INFO     | __main__:<module>:26 - HW I2C Port 

In [4]:
ret, slaves = dev.i2c_find_slave(port=1)
print(slaves)

['0x38', '0x5D']
